In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization

In [2]:
image_directory = 'E:\B. RFMiD_Challenge_Dataset\RFMiD_Challenge_Dataset\Original Images\Training Set\\'

#Now let us read metadata to get our Y values (multiple lables)
df = pd.read_csv('E:\B. RFMiD_Challenge_Dataset\RFMiD_Challenge_Dataset\Groundtruths\RFMiD_Training_Labels.csv', dtype={'ID':str})

In [27]:
print(df.head())     # printing first five rows of the file
print(df.columns)

  ID  Disease_Risk  DR  ARMD  MH  DN  MYA  BRVO  TSLN  ERM  ...  AION  PT  RT  \
0  1             1   1     0   0   0    0     0     0    0  ...     0   0   0   
1  2             1   1     0   0   0    0     0     0    0  ...     0   0   0   
2  3             1   1     0   0   0    0     0     0    0  ...     0   0   0   
3  4             1   0     0   1   0    0     0     0    0  ...     0   0   0   
4  5             1   1     0   0   0    0     0     0    0  ...     0   0   0   

   RS  CRS  EDN  RPEC  MHL  RP  OTHER  
0   0    0    0     0    0   0      0  
1   0    0    0     0    0   0      0  
2   0    0    0     0    0   0      0  
3   0    0    0     0    0   0      0  
4   0    0    0     0    0   0      0  

[5 rows x 30 columns]
Index(['ID', 'Disease_Risk', 'DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN',
       'ERM', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST',
       'AION', 'PT', 'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'OTHER'],
      dtype='o

In [28]:
df = df.iloc[:1920]  #Loading only first 1920 datapoints for memory reasons 
#Need to read images using the tag from metadata.
#Otherwise, if read directly from the folder then images may not correspond to 
#the metadata from the csv file. 

In [29]:
SIZE = 200
X_dataset = []  
for i in tqdm(range(df.shape[0])):
    img = image.load_img(image_directory +df['ID'][i]+'.png', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
    
X = np.array(X_dataset)

100%|██████████████████████████████████████████████████████████████████████████████| 1920/1920 [02:40<00:00, 11.95it/s]


In [30]:
print(df['ID'][500])

501


In [54]:
y = np.array(df.drop(['ID', 'Disease_Risk'], axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.3)

In [55]:
model = Sequential()

In [56]:
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(28, activation='sigmoid'))

In [57]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 196, 196, 16)      1216      
_________________________________________________________________
batch_normalization_16 (Batc (None, 196, 196, 16)      64        
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 98, 98, 16)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 94, 94, 32)        12832     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 47, 47, 32)       

In [58]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/10
21/21 [==============================] - 38s 2s/step - loss: 3.3961 - accuracy: 0.1868 - val_loss: 3.2690 - val_accuracy: 0.3646
Epoch 2/10
21/21 [==============================] - 39s 2s/step - loss: 3.2293 - accuracy: 0.1659 - val_loss: 3.2100 - val_accuracy: 0.0503
Epoch 3/10
21/21 [==============================] - 38s 2s/step - loss: 3.1514 - accuracy: 0.2545 - val_loss: 3.0455 - val_accuracy: 0.1753
Epoch 4/10
21/21 [==============================] - 38s 2s/step - loss: 3.0632 - accuracy: 0.3512 - val_loss: 3.0918 - val_accuracy: 0.1198
Epoch 5/10
21/21 [==============================] - 37s 2s/step - loss: 3.0365 - accuracy: 0.2671 - val_loss: 3.0787 - val_accuracy: 0.0642
Epoch 6/10
21/21 [==============================] - 40s 2s/step - loss: 2.9517 - accuracy: 0.3185 - val_loss: 3.1577 - val_accuracy: 0.1111
Epoch 7/10
21/21 [==============================] - 42s 2s/step - loss: 2.9094 - accuracy: 0.2850 - val_loss: 3.0443 - val_accuracy: 0.3108
Epoch 8/10
21/21 [==